In [1]:
from collections import defaultdict, Counter
import datetime as dt
import emoji
from icecream import ic
from IPython.display import display_html 
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import regex as rex
import shutil
from string import punctuation
import time
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords

import spacy

from sklearn.feature_extraction.text import TfidfTransformer, \
CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline

import textacy.preprocessing as tprep
from textacy.extract import keyword_in_context

In [2]:
to_csv_flag = False
# Establish working directories for saving dataframes as CSV files.

'''Dir nav citation:
https://softhints.com/python-change-directory-parent/
'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\599_team_project\deliverables
C:\Users\acarr\Documents\GitHub\599_team_project


In [3]:
# Get current date/time to append to file name string.

today = dt.datetime.today()
today= str(today)
today = today.replace(':', '-').replace('.', '').replace(' ', '_')
print(today)
print(type(today))

2023-07-30_10-11-18217716
<class 'str'>


In [4]:
# change `data_large_location` to the location of the folder on your machine.
data_large_location = 'data_large'
ref_docs_location = 'ref_docs'

file_in_name01 = 'news-05.csv'

#file_in_path01 = os.path.join(up1_dir, data_large_location, file_in_name01)
file_in_path01 = os.path.join(up1_dir, file_in_name01)

print(f'CSV file in 1 path: {file_in_path01}')

CSV file in 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\news-05.csv


### Review dataframe

Read in data from CSV, check resulting dataframe shape, and display first several records.

In [5]:
slct_tbl_full_df01 = pd.read_csv(file_in_path01)

In [6]:
print(f'Dataframe shape: {slct_tbl_full_df01.shape}')
display(slct_tbl_full_df01.head())

Dataframe shape: (36405, 16)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_distilbert_cat,sentiment_distilbert_prob,sentiment_distilbert,sentiment_bert_cat,sentiment_bert_prob,sentiment_bert,sentiment_roberta_cat,sentiment_roberta_prob,sentiment_roberta,sentiment_roberta_m_cat,sentiment_roberta_m_prob,sentiment_roberta_m
0,2,"{'neg': 0.041, 'neu': 0.888, 'pos': 0.072, 'co...",0.8658,positive,negative,0.990520,-0.990520,positive,0.606353,0.606353,positive,0.998166,0.998166,neutral,0.607053,0.0
1,3,"{'neg': 0.043, 'neu': 0.85, 'pos': 0.107, 'com...",0.9769,positive,positive,0.981083,0.981083,positive,0.600210,0.600210,positive,0.996661,0.996661,neutral,0.642974,0.0
2,5,"{'neg': 0.02, 'neu': 0.817, 'pos': 0.163, 'com...",0.9900,positive,negative,0.986331,-0.986331,positive,0.558369,0.558369,positive,0.995694,0.995694,neutral,0.641572,0.0
3,6,"{'neg': 0.016, 'neu': 0.888, 'pos': 0.096, 'co...",0.9788,positive,positive,0.993403,0.993403,positive,0.622444,0.622444,positive,0.996823,0.996823,neutral,0.598011,0.0
4,7,"{'neg': 0.061, 'neu': 0.853, 'pos': 0.085, 'co...",0.6697,positive,positive,0.910206,0.910206,positive,0.632759,0.632759,positive,0.998417,0.998417,neutral,0.542974,0.0


In [7]:
display(slct_tbl_full_df01.describe())

,text_id,sentiment_vader_compound,sentiment_distilbert_prob,sentiment_distilbert,sentiment_bert_prob,sentiment_bert,sentiment_roberta_prob,sentiment_roberta,sentiment_roberta_m_prob,sentiment_roberta_m
count,36405.000000,36405.000000,36405.000000,36405.000000,36405.000000,36405.000000,36405.000000,36405.000000,36405.000000,36405.000000
mean,22393.006867,0.388630,0.949661,-0.501670,0.600562,0.597694,0.989215,0.379269,0.632488,-0.008617
std,16159.115108,0.842946,0.098727,0.812372,0.024774,0.063636,0.037249,0.914392,0.112328,0.417404
min,2.000000,-1.000000,0.500017,-0.999799,0.500383,-0.625768,0.501201,-0.999511,0.370086,-0.957942
25%,10616.000000,-0.739100,0.964211,-0.996241,0.585972,0.585965,0.992663,-0.986484,0.543655,0.000000
50%,20320.000000,0.963800,0.991151,-0.986197,0.602639,0.602637,0.997089,0.995211,0.612641,0.000000
75%,29543.000000,0.995600,0.996804,-0.535706,0.617610,0.617606,0.998357,0.998237,0.699804,0.000000
max,71592.000000,1.000000,0.999799,0.999794,0.676232,0.676232,0.999511,0.998937,0.990279,0.990279


In [8]:
slct_tbl_full_df01a = slct_tbl_full_df01.loc[slct_tbl_full_df01['sentiment_bert'] > .95]

In [9]:
print(f'Dataframe shape: {slct_tbl_full_df01a.shape}')
display(slct_tbl_full_df01a.head())

Dataframe shape: (0, 16)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_distilbert_cat,sentiment_distilbert_prob,sentiment_distilbert,sentiment_bert_cat,sentiment_bert_prob,sentiment_bert,sentiment_roberta_cat,sentiment_roberta_prob,sentiment_roberta,sentiment_roberta_m_cat,sentiment_roberta_m_prob,sentiment_roberta_m


In [10]:
# Set path to write CSV file to.

file_out_name01 = f'data_sa_w_sw_half1_{today}.csv'
file_out_name02 = f'data_sa_w_sw_half2_{today}.csv'

file_out_path01 = os.path.join(up1_dir, file_out_name01)
file_out_path02 = os.path.join(up1_dir, file_out_name02)

print(f'CSV file out 1 path: {file_out_path01}')
print(f'CSV file out 2 path: {file_out_path02}')

export_col_names_lst = ['text_id',
                        'sentiment_bert',
                       ]

# Write pandas dataframe to CSV; save locally.

len_half = int(round(len(slct_tbl_full_df01)/2,0))
print(len_half)

if to_csv_flag == False:
    pass
else:
    slct_tbl_full_df01[export_col_names_lst][:len_half]\
    .to_csv(file_out_path01, index=False)
    slct_tbl_full_df01[export_col_names_lst][len_half:]\
    .to_csv(file_out_path02, index=False)

print(slct_tbl_full_df01.shape)
display(slct_tbl_full_df01.sample(5))

CSV file out 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_sa_w_sw_half1_2023-07-30_10-11-18217716.csv
CSV file out 2 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_sa_w_sw_half2_2023-07-30_10-11-18217716.csv
18202
(36405, 16)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_distilbert_cat,sentiment_distilbert_prob,sentiment_distilbert,sentiment_bert_cat,sentiment_bert_prob,sentiment_bert,sentiment_roberta_cat,sentiment_roberta_prob,sentiment_roberta,sentiment_roberta_m_cat,sentiment_roberta_m_prob,sentiment_roberta_m
12605,14484,"{'neg': 0.063, 'neu': 0.806, 'pos': 0.131, 'co...",0.9861,positive,negative,0.998402,-0.998402,positive,0.632225,0.632225,positive,0.994419,0.994419,neutral,0.595654,0.000000
5557,7016,"{'neg': 0.039, 'neu': 0.811, 'pos': 0.149, 'co...",0.9971,positive,positive,0.994050,0.994050,positive,0.605657,0.605657,positive,0.998660,0.998660,positive,0.855341,0.855341
1361,2140,"{'neg': 0.063, 'neu': 0.926, 'pos': 0.011, 'co...",-0.9568,negative,negative,0.998662,-0.998662,positive,0.602851,0.602851,positive,0.823022,0.823022,negative,0.505849,-0.505849
10260,11799,"{'neg': 0.059, 'neu': 0.784, 'pos': 0.157, 'co...",0.9592,positive,negative,0.994032,-0.994032,positive,0.610840,0.610840,positive,0.991535,0.991535,negative,0.534874,-0.534874
4326,5771,"{'neg': 0.069, 'neu': 0.874, 'pos': 0.057, 'co...",-0.3990,negative,negative,0.999060,-0.999060,positive,0.613365,0.613365,negative,0.992534,-0.992534,negative,0.534672,-0.534672
